In [1]:
!pip install -r requirements.txt

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [2]:
import os
import numpy as np
import pandas as pd
import math 
import calendar
import warnings
from itertools import product
import seaborn as sns
from time import time
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt
from matplotlib.dates import date2num

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
warnings.filterwarnings('ignore')

import xgboost as xgb
import lightgbm as lgb

In [3]:
df = pd.read_csv("./prepared_sales_data.csv")

In [4]:
df = df.rename(columns={"total_sales": 'sales'})
df

,customer_id,product_id,day,month,year,date,sales
0,1000178,0A4G5LZWCP,15,9,2017,2017-09-15,11
1,1000178,0A4G5LZWCP,25,8,2017,2017-08-25,5
2,1000178,0A4G5LZWCP,27,3,2017,2017-03-27,1
3,1000178,0A4G5LZWCP,28,3,2017,2017-03-28,2
4,1000178,0A4G5LZWCP,28,8,2017,2017-08-28,1
...,...,...,...,...,...,...,...
299570,9998153,VD51ZQPY9D,12,9,2019,2019-09-12,20
299571,9998153,XZ6UVEYK40,5,5,2019,2019-05-05,3
299572,9998153,XZ6UVEYK40,19,7,2019,2019-07-19,2
299573,9998153,XZ6UVEYK40,20,7,2019,2019-07-20,1


In [5]:
def sales_D_M_or_Y(df, time=["daily", 'monthly','yearly']):
    df = df.copy()
    if time == "daily":
        df = df.groupby('date')['sales'].sum().reset_index()
    elif time == "monthly":
        df = df.groupby('month')['sales'].sum().reset_index()
    else:
        df = df.groupby('year')['sales'].sum().reset_index()
    return df

In [6]:
# df_day = sales_D_M_or_Y(df, "daily")
# df_day.head()

In [7]:
# df_month= sales_D_M_or_Y(df, "monthly")
# df_month.head()

In [8]:
# df_day.plot(figsize=(20, 8))

In [9]:
df1 = df.copy()
df1

,customer_id,product_id,day,month,year,date,sales
0,1000178,0A4G5LZWCP,15,9,2017,2017-09-15,11
1,1000178,0A4G5LZWCP,25,8,2017,2017-08-25,5
2,1000178,0A4G5LZWCP,27,3,2017,2017-03-27,1
3,1000178,0A4G5LZWCP,28,3,2017,2017-03-28,2
4,1000178,0A4G5LZWCP,28,8,2017,2017-08-28,1
...,...,...,...,...,...,...,...
299570,9998153,VD51ZQPY9D,12,9,2019,2019-09-12,20
299571,9998153,XZ6UVEYK40,5,5,2019,2019-05-05,3
299572,9998153,XZ6UVEYK40,19,7,2019,2019-07-19,2
299573,9998153,XZ6UVEYK40,20,7,2019,2019-07-20,1


In [10]:
df1.index = df1.date
df1.index = pd.to_datetime(df1.index)

In [11]:
def create_date_features(df):
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    df['day_of_month'] = df.index.day
    df['day_of_year'] = df.index.dayofyear
    df['week_of_year'] = df.index.isocalendar().week
    df["week_of_year"] = df["week_of_year"].astype(np.int64)
    df['day_of_week'] = df.index.dayofweek
    df['year'] = df.index.year
    df["is_wknd"] = df.index.weekday // 4
    df['is_month_start'] = df.index.is_month_start.astype(int)
    df['is_month_end'] = df.index.is_month_end.astype(int)
    return df

df1 = create_date_features(df1)


In [12]:
def add_diff(df, diff_column):
    df['prev_sales'] = df[diff_column].shift(1)
    df['diff']= (df[diff_column] - df['prev_sales'])
    return df
df1 = add_diff(df1, 'sales')

In [13]:
df1.index.names = ['id']

In [14]:
#lag features
#first we add random noise to our dataset
def random_noise(dataframe):
    return np.random.normal(scale=1.6, size=(len(dataframe),))

#then we define lag features function with added noise
def lag_features_v2(dataframe, lags):
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["customer_id", "product_id"])['sales'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe

df1 = lag_features_v2(df1, [91, 98, 105, 182, 364])

In [15]:
#rolling mean features (with random noise)
def roll_mean_features_v2(dataframe, windows):
    for window in windows:
        dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["customer_id", "product_id"])['sales'].transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(dataframe)
    return dataframe

df1 = roll_mean_features_v2(df1, [365])

In [16]:
# #Exp. weighted mean features
# def ewm_features_v2(dataframe, alphas, lags):
#     for alpha in alphas:
#         for lag in lags:
#             dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = dataframe.groupby([
#                 "customer_id", "product_id"])['sales'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
#     return dataframe

# alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
# lags = [91, 98, 105, 112, 180, 270, 365, 546, 728]
# df1 = ewm_features_v2(df1, alphas, lags)

In [17]:
df_all = pd.get_dummies(df1, columns=['customer_id', 'product_id', 'day_of_week', 'month'])

# log conversion of sales 
# df_all['sales'] = np.log1p(df_all["sales"].values)

In [18]:
df_all

day  year        date  sales  quarter  day_of_month  day_of_year  week_of_year  is_wknd  is_month_start  is_month_end  prev_sales  diff  sales_lag_91  sales_lag_98  sales_lag_105  sales_lag_182  sales_lag_364  sales_roll_mean_365  customer_id_1000178  customer_id_1000692  customer_id_1000821  customer_id_1004714  customer_id_1005470  customer_id_1005675  customer_id_1005984  customer_id_1022186  customer_id_1025480  customer_id_1027827  customer_id_1040887  customer_id_1050688  customer_id_1053336  customer_id_1057267  customer_id_1063867  customer_id_1072141  customer_id_1086499  customer_id_1090310  customer_id_1091342  customer_id_1096485  customer_id_1097665  customer_id_1098203  customer_id_1099750  customer_id_1101644  customer_id_1103275  customer_id_1105130  customer_id_1107922  customer_id_1109950  customer_id_1110543  customer_id_1117856  customer_id_1117942  customer_id_1119781  customer_id_1120148  customer_id_1123623  customer_id_1133818  customer_id_1138377  \
id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
2017-09-15   15  2017  2017-09-15     11        3            15          258            37        1               0             0         NaN   NaN           NaN           NaN            NaN            NaN            NaN                  NaN                 True                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False   
2017-08-25   25  2017  2017-08-25      5        3            25          237            34        1               0             0        11.0  -6.0           NaN           NaN            NaN            NaN            NaN                  NaN                 True                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False                False   
2017-03-

In [19]:
#Creating model
#defining custom cost function
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val


def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(preds, labels)
    return 'SMAPE', smape_val, False

In [20]:
train = df_all.loc[(df_all["date"] < "2019-01-01"), :]
# last one month validation
val = df_all.loc[(df_all["date"] >= "2019-01-01") & (df_all["date"] < "2019-04-01"), :]
cols = [col for col in train.columns if col not in ['date', "sales", "year"]]
Y_train = train['sales']
X_train = train[cols]
Y_val = val['sales']
X_val = val[cols]

In [ ]:
lgb_params = {'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 1500,
              'early_stopping_rounds': 300,
              'nthread': -1}
lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)
model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbval],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  feval=lgbm_smape,
                  verbose_eval=100)

In [ ]:
y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)

In [ ]:
smape(y_pred_val, Y_val)

In [ ]:
train = df_all.loc[~df_all.sales.isna()]
Y_train = train['sales']
X_train = train[cols]
test = df_all.loc[df_all.sales.isna()]
X_test = test[cols]

In [ ]:
lgb_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'nthread': -1,
              "num_boost_round": model.best_iteration}


lgbtrain_all = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)

model = lgb.train(lgb_params, lgbtrain_all, num_boost_round=model.best_iteration)

test_preds = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
#draw importance plot
def plot_lgb_importances(model, plot=False, num=10):
    gain = model.feature_importance('gain')
    feat_imp = pd.DataFrame({'feature': model.feature_name(),
                             'split': model.feature_importance('split'),
                             'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
    if plot:
        plt.figure(figsize=(10, 10))
        sns.set(font_scale=1)
        sns.barplot(x="gain", y="feature", data=feat_imp[0:25])
        plt.title('feature')
        plt.tight_layout()
        plt.show()
    else:
        print(feat_imp.head(num))
plot_lgb_importances(model, num=30, plot=True)

In [ ]:
test_preds = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
def product_forecast(only_customer=False, only_customer_plot=False,
                     only_item=False, only_item_plot=False,
                     customer_item=False, customer_item_plot=False, 
                     customer="S0028", item="P0008", period="MS"):

    if only_customer:
        df_test_preds = pd.DataFrame(test_preds, columns=["sales"])
        df_all["sales"].fillna(df_test_preds["sales"], inplace=True)
        df_item = df_all[df_all[customer]==1]
        df_item['sales'] = df_item['sales']
        df_item.set_index("date", inplace=True)
        df_item = df_item.resample(period).sum()
        df_item = df_item["sales"]

        if only_customer_plot:
         fig, ax = plt.subplots(figsize=(12, 6))
         ax.plot(df_item.index, df_item, color="r")
         ax.axvspan(date2num(datetime.datetime(2019, 10, 1)), date2num(datetime.datetime(2019, 10, 30)),
                    label="Prediction", color="navy", alpha=0.3)
         ax.legend()
         ax.set_xlabel("Date")
         ax.set_ylabel('Amount')
         ax.set_title(f"{customer}'s sales forecast for all items")
         plt.grid(axis="both")
         plt.tight_layout()
         plt.show()

    if only_item:
        df_test_preds = pd.DataFrame(test_preds, columns=["sales"])
        df_all["sales"].fillna(df_test_preds["sales"], inplace=True)
        df_item = df_all[df_all[item]==1]
        df_item['sales'] = df_item['sales']
        df_item.set_index("date", inplace=True)
        df_item = df_item.resample(period).sum()
        df_item = df_item["sales"]

        if only_item_plot:
         fig, ax = plt.subplots(figsize=(12, 6))
         ax.plot(df_item.index, df_item, color="r")
         ax.axvspan(date2num(datetime.datetime(2019, 10, 1)), date2num(datetime.datetime(2019, 10, 30)),
                    label="Prediction", color="navy", alpha=0.3)
         ax.legend()
         ax.set_xlabel("Date")
         ax.set_ylabel('Amount')
         ax.set_title(f"{item}'s sales forecast for all customers")
         plt.grid(axis="both")
         plt.tight_layout()
         plt.show()

    if customer_item:
        df_test_preds = pd.DataFrame(test_preds, columns=["sales"])
        df_all["sales"].fillna(df_test_preds["sales"], inplace=True)
        df_item = df_all[(df_all[customer]==1) & (df_all[item]==1)]
        df_item['sales'] = df_item['sales']
        df_item.set_index("date", inplace=True)
        df_item = df_item.resample(period).sum()
        df_item = df_item["sales"]

        if customer_item_plot:
         fig, ax = plt.subplots(figsize=(12, 6))
         ax.plot(df_item.index, df_item, color="r")
         ax.axvspan(date2num(datetime.datetime(2019, 10, 1)), date2num(datetime.datetime(2019, 10, 30)),
                    label="Prediction", color="navy", alpha=0.3)
         ax.legend()
         ax.set_xlabel("Date")
         ax.set_ylabel('Amount')
         ax.set_title(f"{item}'s sales forecast at {customer}")
         plt.grid(axis="both")
         plt.tight_layout()
         plt.show()

    return df_item